In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
#week_day_Jan_list = [2, 9, 16, 23, 30]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [14]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [16]:
x[:,:2]

matrix([[  647382.36325344,   640237.8298605 ],
        [  678225.89645663,   661148.64361712],
        [ 1196529.98249006,  1188788.5281001 ],
        [ 1268351.55896686,  1246949.25519545],
        [  309510.49283983,   301925.77575403],
        [  396527.35793771,   380762.99913631],
        [  782394.99884797,   749910.29551765],
        [  764039.77542994,   760696.86046897],
        [  682585.49002355,   688203.2820016 ],
        [  675105.70640671,   707965.89843347],
        [  733622.38703719,   699727.69566146],
        [  631918.2823594 ,   628820.51443439],
        [  179092.60871588,   180575.80381719],
        [  185537.7439059 ,   183369.4048835 ],
        [  303278.4049973 ,   284382.39444304],
        [  262923.53587039,   275200.93253566],
        [  382453.08046721,   398597.64654216],
        [  395896.30947978,   409521.93802392],
        [  608784.21063278,   640370.55357122],
        [  565458.25052227,   575998.31370421],
        [  935145.17524493,   921911.505

In [17]:
np.size(A,0), np.size(A,1)

(24, 140)

In [18]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [6e-02, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.13831211e+07  0.00000000e+00  4.72e+03 1.73e+03  1.00e+06     0s
   1  -1.17056634e+07 -4.21105843e+06  1.66e+03 6.08e+02  3.85e+05     0s
   2  -1.16858876e+07 -1.19473834e+07  2.45e+02 8.99e+01  9.69e+04     0s
   3  -1.19415919e+07 -1.23941179e+07  1.26e+00 4.64e-01  8.5

In [19]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [20]:
lam_list

[2.3558992546909246e-10,
 7.137960343510169e-07,
 141791.05210124308,
 6.939979639114481e-07,
 7.92399508402924e-07,
 420997.7653718165,
 7.223285708468381e-07,
 5.823344350310582e-10,
 1.4586554606019774e-11,
 257435.43755973893,
 1.4586566475692642e-11,
 1.4586567210367992e-11,
 1.4586737811078684e-11,
 1.4684819068224633e-11,
 406763.0778631107,
 2.01011717998532e-11,
 8.435113075200512e-08,
 45822.45450274898,
 32824.83835064018,
 6.545739356348008e-07,
 120823.95260536138,
 186500.76133956813,
 451830.70113400085,
 1.5900413593083833e-07,
 8.601420449887154e-05,
 0.0002159303933608192,
 48787.09504134045,
 361878.9958436341,
 351813.1641590776,
 2.010117166698814e-11,
 1.962906025240285e-07,
 5.092828778849529e-07,
 17116.713214483374,
 3.98133701856849e-09,
 24346.487212203116,
 121872.87712817198,
 2.0101172084859934e-11,
 1.4036689784224896e-07,
 79895.51249018477,
 4.091356616480282e-07,
 236239.70822791004,
 9.591551491818342e-07,
 161770.39592472685,
 2.010117218147463e-11,
